In [ ]:
import os, sys
import numpy as np
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
import spectral

In [ ]:
from spectral.algorithms import spectral_angles

In [ ]:
x = np.random.rand(1, 10, 25)
endmember = np.random.rand(5, 25)
spectral_angles(x, endmember).shape

# Data Preprocessing

## Calibration 

$\begin{align}
    C_i = 100 * \frac{R_i - D_r}{W_r - D_r}
\end{align}$

where $C_i$ is the calibrated image, $R_i$ note raw image and the $W_r$ and $D_r$ represents the white and dark reference image, respectively.


In [68]:
# W_r y D_r es una imagen
def calibrate(img, w_r, d_r):
    if not(w_r.shape == d_r.shape == img.shape):
        assert('Dimensionality error')
    
    return 100  * (img - d_r) / (w_r - d_r)

False

# Normalize data
$\begin{align}
    P'_i = \frac{P_i - P_{min}}{P_{max} - P_{min}}
\end{align}$

where $P'_i$ is the normalized pixel value, $P_i$ the reflectance of the pixel, $P_{min}$ and $P_{max}$ is the minimum and maximum reflectance value, respectively.

In [72]:
def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

0.03031085034079306

# Sampling Interval Analysis

Selected band equidistant...

$\begin{align}
    Sampling Interval (nm) = \frac{\lambda_{max} - \lambda_{min}}{N_{\lambda}}
\end{align}$

where $\lambda_{max} - \lambda_{min}$ is the difference between the mamum and minimum wavelength and $N_{\lambda}$ is the number of band captured by the sensor.

In [ ]:
def sampling_interval(lambda_min, lambda_max, n_spectral_bands):
    '''
        Param:
        -----
            lambda_min (int): minimum wavelenght
            lambda_max (int): maximum wavelenght
            n_spectral_bands (int): number of spectral bands captured by the sensor
    '''
    return (lambda_max - lambda_min) / n_spectral_bands

# Dataset reduction

**Spectral Angle Mapper**

$[\alpha = cos^{-1}\left ( \frac{\sum_{i = 1}^{nb} t_{i} r_{i}}{(\sum_{i = 1}^{nb} t_{i}^2)^{\frac{1}{2}} (\sum_{i = 1}^{nb} r_{i}^2)^{\frac{1}{2}}} \right )]$

where

* $\alpha$ = spectral angle between the standard and the spectral curve of the pixel
* $nb$ = number of spectral channels
* $t$ = vector of spectral response of the standard
* $r$ = the spectral response vector of the analyzed pixel

In [ ]:
from sklearn.cluster import KMeans
from spectral.algorithms import spectral_angles

# example, 12 pixels with 125 bands
X = np.random.rand(12, 125)

kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
centroid = kmeans.cluster_centers_

spectral_angles(X[0:1].T, centroid[0:1])

In [62]:
x = X
x = x[np.newaxis, :]
spectral_angles(x, centroid).shape

(1, 12, 4)

For each centroid, only the 10 most similar pixels are selected, having a total of 1000 pixels per class (100 centroids ×10 pixels). Thus, the reduced dataset is
intended to avoid the inclusion of redundant information in the training of the supervised classifier...

In [ ]:
centroid.shape

In [ ]:
n_cluster = 2

X = np.array([[1, 2.5], [1, 4.1], [1, 0.1],
        [10, 2.1], [10, 4.9], [10, 0]])

kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(X)
centroid = kmeans.cluster_centers_
for x in X:
    plt.scatter(x=x[0], y=x[1], alpha=.2)

for x in centroid:
    plt.scatter(x=x[0], y=x[1], marker='*')

plt.show()

# Spectral Angle Mapper

$[\alpha = cos^{-1}\left ( \frac{\sum_{i = 1}^{nb} t_{i} r_{i}}{(\sum_{i = 1}^{nb} t_{i}^2)^{\frac{1}{2}} (\sum_{i = 1}^{nb} r_{i}^2)^{\frac{1}{2}}} \right )]$

where

* $\alpha$ = spectral angle between the standard and the spectral curve of the pixel
* $nb$ = number of spectral channels
* $t$ = vector of spectral response of the standard
* $r$ = the spectral response vector of the analyzed pixel

In [ ]:
X = ["a", "b", "c", "d"]
kf = KFold(n_splits=4)
for train, test in kf.split(X):
    print("%s %s" % (train, test))